In [88]:
%%time
from time import sleep
from random import randint

n_pages = 0

for page in range(0,900):
    n_pages += 1
    sapo_url = 'https://casa.sapo.pt/Venda/Apartamentos/?sa=11&lp=10000&or=10'+'&pn='+str(page)
    r = get(sapo_url, headers=headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_="searchResultProperty")
    if house_containers != []:
        for container in house_containers:
            
            # Price            
            price = container.find_all('span')[2].text
            if price == 'Contacte Anunciante':
                price = container.find_all('span')[3].text
                if price.find('/') != -1:
                    price = price[0:price.find('/')-1]
            if price.find('/') != -1:
                price = price[0:price.find('/')-1]
            
            price_ = [int(price[s]) for s in range(0,len(price)) if price[s].isdigit()]
            price = ''
            for x in price_:
                price = price+str(x)
            prices.append(int(price))

            # Zone
            location = container.find_all('p', class_="searchPropertyLocation")[0].text
            location = location[7:location.find(',')]
            zone.append(location)

            # Title
            name = container.find_all('span')[0].text
            titles.append(name)

            # Status
            status = container.find_all('p')[5].text
            condition.append(status)

            # Area
            m2 = container.find_all('p')[9].text
            if m2 != '-':
                m2 = m2.replace('\xa0','')
                m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                areas.append(m2)
                
            else:
                m2 = container.find_all('p')[7].text
                if m2 != '-':
                    m2 = m2.replace('\xa0','')
                    m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                    areas.append(m2)
                else:
                    areas.append(m2)

            # Creation date
     #       date = pd.to_datetime(container.find_all('div', class_="searchPropertyDate")[0].text[21:31])
     #       created.append(date)

            # Description
            desc = container.find_all('p', class_="searchPropertyDescription")[0].text[7:-6]
            descriptions.append(desc)

            # url
            link = 'https://casa.sapo.pt/' + container.find_all('a')[0].get('href')[1:-6]
            urls.append(link)

            # image
            img = str(container.find_all('img')[0])
            img = img[img.find('data-original_2x=')+18:img.find('id=')-2]
            thumbnails.append(img)
    else:
        break
    
    sleep(randint(1,2))
    
print('You scraped {} pages containing {} properties.'.format(n_pages, len(titles)))






cols = ['Title', 'Zone', 'Price', 'Size (m²)', 'Status', 'Description', 'URL', 'Image']

lisboa = pd.DataFrame({'Title': titles,
                           'Price': prices,
                           'Size (m²)': areas,
                           'Zone': zone,
      #                     'Date': created,
                           'Status': condition,
                           'Description': descriptions,
                           'URL': urls,
                           'Image': thumbnails})[cols]

lisboa.to_excel('lisboa_raw.xls')

# lisboa = pd.read_excel('lisboa_raw.xls')

ValueError: invalid literal for int() with base 10: ''

In [2]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [4]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [5]:
sapo = "https://casa.sapo.pt/Venda/Apartamentos/?sa=11&or=10"
response = get(sapo, headers=headers)

In [6]:
response

<Response [200]>

In [7]:
print(response)

<Response [200]>


In [8]:
print(response.text[:1000])



<!DOCTYPE html>

<html lang="pt">
<head><title>
	Casas para Venda, Apartamentos Com Ofertas recentes no Distrito de Lisboa, CASA SAPO - Portal Nacional de Imobiliário
</title><meta name="author" content="CASA SAPO - Portal Nacional de Imobiliário - Janela Digital SA" />
    <meta name="application-name" content="CASA SAPO - Portal Nacional de Imobiliário" data-copyright="Janela Digital SA" data-generated-time="28/05/2020 19:05" />
    
<meta name="content-language" content="pt" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<meta name="description" content="Casas para Venda, 29949 Apartamentos Com Ofertas recentes no Distrito de Lisboa, Deseja comprar casa? No maior Portal Imobiliário Nacional temos milhares de apartamentos e moradias em Lisboa, no Porto e por todo o país." />
<meta name="keywords" content="Casas para Venda, Apartamentos Com Ofertas recentes no Distrito de Lisboa, venda, compra, comprar, casas, imóveis, apartamentos, moradias


In [9]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [10]:
html_soup


<!DOCTYPE html>

<html lang="pt">
<head><title>
	Casas para Venda, Apartamentos Com Ofertas recentes no Distrito de Lisboa, CASA SAPO - Portal Nacional de Imobiliário
</title><meta content="CASA SAPO - Portal Nacional de Imobiliário - Janela Digital SA" name="author"/>
<meta content="CASA SAPO - Portal Nacional de Imobiliário" data-copyright="Janela Digital SA" data-generated-time="28/05/2020 19:05" name="application-name"/>
<meta content="pt" name="content-language"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="Casas para Venda, 29949 Apartamentos Com Ofertas recentes no Distrito de Lisboa, Deseja comprar casa? No maior Portal Imobiliário Nacional temos milhares de apartamentos e moradias em Lisboa, no Porto e por todo o país." name="description"/>
<meta content="Casas para Venda, Apartamentos Com Ofertas recentes no Distrito de Lisboa, venda, compra, comprar, casas, imóveis, apartamentos, moradias, terrenos, porto, lisboa" name="keywords"/>

In [11]:
house_containers = html_soup.find_all('div', class_="searchResultProperty")

In [12]:
house_containers

[<div class="searchResultProperty item" data-uid="1352a0c5-d5f1-4151-a22a-2b4baec31930">
 <script type="application/ld+json">{"@context":"http://schema.org","@type":"Offer","image":"http://media.casasapo.pt/Z640x480/Wnone/S6/C603607/P200261245/Tphoto/IDefa21fac-396b-45e1-896b-e8f4bdc48e50.jpg","name":"Apartamento T2 Alto da Faia, Lumiar, Lisboa","category":"Apartamentos","description":"T2 remodelado, no Alto da Faia em Telheiras para Arrendamento.Andar no 7º e ultimo andar com vista desafogada. Edifício com 2 elevadores.  Localizado na Rua Prof. Queiros veloso a 7 minutos a pé do Metro de Telheiras , (...)","price":["350 000 €","1 300 €"],"priceCurrency":["€","€"],"availableAtOrFrom":{"@type":"Place","address":{"addressCountry":"PT","addressLocality":"Lisboa","addressRegion":"Lumiar"},"geo":{"@type":"GeoCoordinates","latitude":38.7641390377,"longitude":-9.1710793041}},"seller":{"@context":"http://schema.org","@type":"Person","name":"André Gaspar","image":"http://casa.sapo.pt/Themes/Glo

In [13]:
first = house_containers[0]
first.find_all('span')

[<span class="titleG3"></span>,
 <span>
                         Apartamento T2, Alto da Faia, Lumiar, Lisboa
                     </span>,
 <span class="btnContactPVPI" id="MC_PropertyInList_repProperties_btnContactPVPINormal_0" onclick="ShowContactForm('1352a0c5-d5f1-4151-a22a-2b4baec31930', '13', '6', false, false, '0'); return false;" style="z-index: 9999;" title="Contacte Anunciante">Contacte Anunciante</span>,
 <span>350 000 <strong title="Euro">€</strong> / 1 300 <strong title="Euro">€</strong></span>]

In [16]:
var_1 = first.find_all('span')[3].text

In [17]:
var_1

'350\xa0000 €\xa0/\xa01\xa0300 €'

In [18]:
var_1 = var_1.replace('\xa0','')
var_1

'350000 €/1300 €'

In [19]:
var_1 = int(''.join(itertools.takewhile(str.isdigit, var_1)))

In [20]:
print(var_1, type(var_1))

350000 <class 'int'>


In [23]:
# location
location = first.find_all('p', class_="searchPropertyLocation")[0].text

In [24]:
location

'\r\n                    Alto da Faia, Lumiar, Lisboa\r\n                '

In [32]:
location[22:location.find(',')]

'Alto da Faia'

In [38]:
# size of building in m2
first.find_all('p')[7]

<p>86m²</p>

In [47]:
#data about posted time
pd.to_datetime(first.find_all('div', class_='searchPropertyFeatures')[0].text[0:10]

SyntaxError: unexpected EOF while parsing (<ipython-input-47-f30accd734a0>, line 2)

In [60]:
first.find_all('p', class_="searchPropertyDescription")[0].text[22:-23]

'T2 remodelado, no\xa0Alto da Faia em Telheiras para\xa0Arrendamento.Andar no 7º e ultimo andar com vista desafogada. Edifício com 2 elevadores.  Localizado na Rua Prof. Queiros veloso a 7 minutos a pé do Metro de Telheiras , '

In [82]:
#getting all the links
for url in first.find_all('a'):
    print('https://casa.sapo.pt' + url.get('href'))

https://casa.sapo.pt/comprar-alugar-apartamento-t2-lisboa-lumiar-alto-da-faia-perto-escola,policia,transportes-publicos-tem-vista-para-cidade,estacionamento,escolas,piscina-1352a0c5-d5f1-4151-a22a-2b4baec31930.html
https://casa.sapo.pt/comprar-alugar-apartamento-t2-lisboa-lumiar-alto-da-faia-perto-escola,policia,transportes-publicos-tem-vista-para-cidade,estacionamento,escolas,piscina-1352a0c5-d5f1-4151-a22a-2b4baec31930.html
https://casa.sapo.pt/comprar-alugar-apartamento-t2-lisboa-lumiar-alto-da-faia-perto-escola,policia,transportes-publicos-tem-vista-para-cidade,estacionamento,escolas,piscina-1352a0c5-d5f1-4151-a22a-2b4baec31930.html
https://casa.sapo.pt/comprar-alugar-apartamento-t2-lisboa-lumiar-alto-da-faia-perto-escola,policia,transportes-publicos-tem-vista-para-cidade,estacionamento,escolas,piscina-1352a0c5-d5f1-4151-a22a-2b4baec31930.html


In [69]:
#add the domain part
'https://casa.sapo.pt' + first.find_all('a')[0].get('href')

'https://casa.sapo.pt/comprar-alugar-apartamento-t2-lisboa-lumiar-alto-da-faia-perto-escola,policia,transportes-publicos-tem-vista-para-cidade,estacionamento,escolas,piscina-1352a0c5-d5f1-4151-a22a-2b4baec31930.html'

In [74]:
p = str(first.find_all('img')[0])
p

'<img alt="Ver Apartamento T2, Alto da Faia, Lumiar, Lisboa, Lumiar em Lisboa" data-error="/Themes/Global/Images/NoPic280x210.jpg.webp" data-src="https://media.casasapo.pt/Z320x240/Wnone/S6/C603607/P200261245/Tphoto/IDefa21fac-396b-45e1-896b-e8f4bdc48e50.jpg.webp" data-src-retina="https://media.casasapo.pt/Z640x480/Wnone/S6/C603607/P200261245/Tphoto/IDefa21fac-396b-45e1-896b-e8f4bdc48e50.jpg.webp" id="MC_PropertyInList_repProperties_thumbImage_0" src="https://media.casasapo.pt/Z320x240/Wnone/S6/C603607/P200261245/Tphoto/IDefa21fac-396b-45e1-896b-e8f4bdc48e50.jpg.webp">\n<a class="photoLayer" href="/comprar-alugar-apartamento-t2-lisboa-lumiar-alto-da-faia-perto-escola,policia,transportes-publicos-tem-vista-para-cidade,estacionamento,escolas,piscina-1352a0c5-d5f1-4151-a22a-2b4baec31930.html" id="MC_PropertyInList_repProperties_hlRealestateDetail_Image_0" title="Ver Apartamento T2, Alto da Faia, Lumiar, Lisboa, Lumiar em Lisboa"></a>\n<div class="photoButtonsBackground"></div>\n</img>'

In [81]:
imgurl = p[p.find('data-src-retina=')+17:p.find('id=')-2]
imgurl 

'https://media.casasapo.pt/Z640x480/Wnone/S6/C603607/P200261245/Tphoto/IDefa21fac-396b-45e1-896b-e8f4bdc48e50.jpg.webp'

In [90]:
# setting up the lists that will form our dataframe with all the results
titles = []
created = []
prices = []
areas = []
zone = []
condition = []
descriptions = []
#urls = []
#thumbnails = []

In [98]:
%%time
from time import sleep
from random import randint

n_pages = 0

for page in range(0,9):
    n_pages += 1
    sapo_url = 'https://casa.sapo.pt/Venda/Apartamentos/?sa=11&or=10'+'&pn='+str(page)
    r = get(sapo_url, headers=headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_="searchResultProperty")
    if house_containers != []:
        for container in house_containers:
            
            # Price            
#            price = container.find_all('span')[2].text
#            if price == 'Contacte Anunciante':
#                price = container.find_all('span')[3].text
#                if price.find('/') != -1:
#                    price = price[0:price.find('/')-1]
#            if price.find('/') != -1:
#                price = price[0:price.find('/')-1]
            
#            price_ = [int(price[s]) for s in range(0,len(price)) if price[s].isdigit()]
#            price = ''
#            for x in price_:
#                price = price+str(x)
#            prices.append(int(price)) 

            # Zone
            location = container.find_all('p', class_="searchPropertyLocation")[0].text
            location = location[22:location.find(',')]
            zone.append(location)

            # Title
            name = container.find_all('span')[0].text
            titles.append(name)

            # Status
            status = container.find_all('p')[5].text
            condition.append(status)

            # Area
            m2 = container.find_all('p')[9].text
            if m2 != '-':
                m2 = m2.replace('\xa0','')
                m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                areas.append(m2)
                
            else:
                m2 = container.find_all('p')[7].text
                if m2 != '-':
                    m2 = m2.replace('\xa0','')
                    m2 = float("".join(itertools.takewhile(str.isdigit, m2)))
                    areas.append(m2)
                else:
                    areas.append(m2)

            # Creation date
     #       date = pd.to_datetime(container.find_all('div', class_="searchPropertyDate")[0].text[21:31])
     #       created.append(date)

            # Description
            desc = container.find_all('p', class_="searchPropertyDescription")[0].text[22:-23]
            descriptions.append(desc)

            # url
            link = 'https://casa.sapo.pt/' + container.find_all('a')[0].get('href')
            urls.append(link)

            # image
    #        img = str(container.find_all('img')[0])
    #        img = img[img.find('data-src-retina='')+17:p.find('id=')-2]
    #        thumbnails.append(img)
    else:
        break
    
    sleep(randint(1,2))
    
print('You scraped {} pages containing {} properties.'.format(n_pages, len(titles)))


You scraped 9 pages containing 1090 properties.
Wall time: 31 s


In [103]:
import pandas as pd

cols = ['Title', 'Size (m²)', 'Zone', 'Status', 'Description']

lisboa = pd.DataFrame({'Title': titles,
      #                     'Price': prices,
                           'Size (m²)': areas,
                           'Zone': zone,
      #                     'Date': created,
                           'Status': condition,
                           'Description': descriptions})[cols]
      #                     'URL': urls,
      #                     'Image': thumbnails})[cols]

#lisboa.to_excel('lisboa_raw.xls')

# lisboa = pd.read_excel('lisboa_raw.xls')

In [104]:
lisboa

,Title,Size (m²),Zone,Status,Description
0,T2 com terraço 22m2 para recuperar na Tapada d...,87,Tapada da Mercês,Por recuperar,Imóvel de 3 ass. com terraço 22m2 situado num ...
1,"T2 Triplex no centro da Parede, com vista mar ...",128,Carcavelos e Parede,Usado,Prédio em betão armado em bom estado de conser...
2,T1 Oeiras - Figueirinha,57,Figueirinha (Oeiras e São Julião Barra),Usado,Apartamento em excelente estado e com muita lu...
3,T3 remodelado e a estrear com terraço 36m2 na ...,136,Ericeira,Recuperado,"Imóvel de 3 ass. com terraço 36m2, totalmente ..."
4,Apartamento T3 com 2 parqueamentos na Quinta N...,154,Quinta Nova,Usado,Apartamento T3 na Quinta Nova em Odivelas Ref...
5,,99,Alto da Faia,Renovado,"T2 remodelado, no Alto da Faia em Telheiras pa..."
6,"Apartamento T3, com vistas desafogadas, na Ven...",91,Centro (Venda Nova),Usado,"Fantástico apartamento T3 (com 4 divisões), pr..."
7,"T3 remodelado a estrear, grandes varandas e ar...",97,Centro (Sacavém),Recuperado,"T3 renovado a estrear, em prédio da década de ..."
8,Apartamento T2 Duplex Travessa da Marquesa/Parede,130,Parede,Usado,Apartamento de 3 assoalhadas divididas em três...
9,Fantástico Apartamento T2 Parqueamento Colinas...,115,Odivelas,Usado,"Fantástico apartamento T2 com parqueamento, co..."


In [105]:
lisboa.to_csv("lisboa_property_data.csv", index=False)